In [1]:
import os
import base64
import langchain
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [2]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

from chatrag.retriever import create_retriever_from_csv, create_retrieval_chain

In [3]:
langchain.debug = True

## Data preparation

In [4]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [5]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"monthly_traffic": "int"},
    llm=retriever_llm
)

# Vectorstore DB

In [6]:
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)

In [7]:
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

# Tooling

In [8]:
calc_tool = load_tools(["llm-math"], llm=OpenAI(temperature=0))

In [9]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence"""

In [10]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    ),
    calc_tool[0]
]

# Prompt templates

In [11]:
# from langchain.agents.structured_chat.base import StructuredChatAgent
# test = StructuredChatAgent.create_prompt(
#     tools,
#     input_variables=["input", "agent_scratchpad"]
# )
# print(test[0].prompt.template)

In [12]:
prefix = """ You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. 

For the first interactions you should collect some user configuration data. This data will restrict the movies to consider.

User Data to collect (mandatory):
    Target genre: Movie genre e.g. fiction, adventure, trhiller...
    Movie overview topic: List of keywords defining the campaign context.

After succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good as you can. To do so, you have access to the following tools:"""

In [13]:
suffix = """Begin! Your first action must ve collect user data and keep it. Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation: ... Thought: ... Action: ..."""


In [14]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": prefix,
        "suffix": suffix
    }

# Chains

In [28]:
# agent_llm_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=1000)
agent_llm_model = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)

In [31]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

# TESTS

## Chat Agent Chain

### Run

In [29]:
langchain.debug = False

In [30]:
memory.chat_memory.messages

[]

In [32]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Hello Aitor! I'm here to help you find the perfect movie to watch. To get started, could you please tell me your preferred movie genre and some keywords that define the type of movie you're in the mood for?

> Finished chain.

A partir de aquí la respuesta:

Hello Aitor! I'm here to help you find the perfect movie to watch. To get started, could you please tell me your preferred movie genre and some keywords that define the type of movie you're in the mood for?


In [33]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Great! Could you please specify the genre you prefer? For example, are you interested in a science fiction movie, a documentary, a thriller set in space, or something else?

> Finished chain.

A partir de aquí la respuesta:

Great! Could you please specify the genre you prefer? For example, are you interested in a science fiction movie, a documentary, a thriller set in space, or something else?


In [34]:
response = general_chat_agent.run(input="A thriller set in space sounds good.")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Perfect! A space thriller it is. Now, could you please provide me with some keywords that you'd like the movie to revolve around? For example, these could be "alien life", "lost in space", "space mission", etc.

> Finished chain.

A partir de aquí la respuesta:

Perfect! A space thriller it is. Now, could you please provide me with some keywords that you'd like the movie to revolve around? For example, these could be "alien life", "lost in space", "space mission", etc.


In [ ]:
print(response)

In [35]:
# TODO: Current issue. Though it identifies what to do, doesn't launch the action tool. 
response = general_chat_agent.run(input="space exploration")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Great! Let's find a thrilling space exploration movie for you. Let me do a quick search.

> Finished chain.

A partir de aquí la respuesta:

Great! Let's find a thrilling space exploration movie for you. Let me do a quick search.


In [36]:
response = general_chat_agent.run(input="search it")
print("\nA partir de aquí la respuesta:\n")
print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search movies",
  "action_input": "thriller space exploration"
}
```

/Users/itortouch/miniconda3/envs/growwer_copilot/lib/python3.11/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='thriller space exploration' filter=None limit=None

Observation: 1. url_media: Interstellar
   - Justification: The movie "Interstellar" is a perfect match for the topic "thriller space exploration". It chronicles the adventures of a group of explorers who use a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage. It has the highest vote average of 8.1 among the listed media.
   - price_sale_dofollow: [Not provided]
   - monthly_traffic: [Not provided]
   - domaing_rating: [Not provided]

2. url_media: Event Horizon
   - Justification: The movie "Event Horizon" is also a good match for the topic. It's about a group of astronauts sent to investigate and salvage the long lost starship "Event Horizon". The ship disappeared mysteriously 7 years before on its maiden voyage and with its return comes even more mystery as the crew discover the real truth behind its disappearance and something even m